In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
import datetime
warnings.filterwarnings("ignore")
np.random.seed(32)

In [ ]:
data=pd.read_csv('../input/covid-world-vaccination-progress/country_vaccinations.csv')
data2=pd.read_csv('../input/covid-19/data/countries-aggregated.csv')

In [ ]:
datan=data[['country','iso_code','date','total_vaccinations_per_hundred','daily_vaccinations']]
datan.sort_values('date',inplace=True)
datan.dropna(inplace=True)
fig = px.scatter(datan, x="total_vaccinations_per_hundred", y='daily_vaccinations', animation_frame="date", animation_group="iso_code",
           hover_name="iso_code", text='iso_code',range_x=[-10,100], range_y=[-100000,1500000])

fig.update_traces(marker=dict(size=32,  color='DarkSlateGrey'))
fig.update_layout(template='plotly_dark')
fig.show()


In [ ]:
name=input('Please enter the country name for the time series analysis :')
datacd=data[data['country']==name]

In [ ]:
# Using simple LSTM code to do some time series predictions :)

datacd.sort_values('date',inplace=True)
lp='total_vaccinations_per_hundred'
datacd.dropna(subset=[lp],inplace=True)
datac=datacd[[lp]]
dates=datacd['date'].values
train,test=train_test_split(datac,test_size=0.2,shuffle=False)
def create_test_train(dataset,look_back=1):
    x=dataset.values[0:len(dataset)-look_back]
    y=dataset.values[look_back:len(dataset)]
    return x,y
look_back = 2
trainX, trainY = create_test_train(train, look_back)
testX, testY = create_test_train(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

# Making the model
model = Sequential()
model.add(LSTM(1000, input_shape=(1, 1)))
model.add(Dense(1000))
model.add(Dense(100))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
fitted_model=model.fit(trainX, trainY, epochs=100, batch_size=8, verbose=2,validation_data=(testX,testY))

In [ ]:
# Plotting the loss of the model
plt.plot(fitted_model.history['loss'])
plt.plot(fitted_model.history['val_loss'])

In [ ]:
trainPredict = fitted_model.model.predict(trainX)
testPredict = fitted_model.model.predict(testX)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=datacd['date'][:len(trainPredict)],y=trainX.flatten(), name="Original data"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=datacd['date'][:len(trainPredict)],y=trainPredict.flatten(), name="Predicted data"),
    secondary_y=True,
)
fig.update_layout(template='plotly_dark')
fig.show()
